In [ ]:
#API libraries
from fredapi import Fred
import yfinance as yf

#Dataframe and time libraries
import pandas as pd
from datetime import datetime

In [ ]:
def fred_data_extract(obs_start,obs_end):

    api_key = '4dda1239a9e25d81bcda77196a642bb1' # Replace with your own FRED API key
    fred = Fred(api_key=api_key)
    
    series_dict = {
        # Some Random Macroeconomic Indicator:Names & Descriptions from FRED to see if the model uses them (if at all)
        'GDP': 'GDP',
        'CPIAUCSL': 'CPI_Inflation',
        'APU0000708111': 'Average_price_of_eggs',
        'APU000072610': 'Average_price_of_electricity',
        'USSLIND': 'Leading_Index_for_US',
        'USREC': 'NBER_Recession_Indicator',
        'PPIACO': 'Producer_Price_Index',
        'T5YIFR': '5-Year_Breakeven_Inflation',
        'NROU': 'Natural_Rate__of_Unemployment_(est.)',
        'CIVPART': 'Civilian_Labor_Force_Participation',
        'BAMLC0A4CBBB': 'BBB_Corporate_Spread',
        # Growth & Business Cycle
        'GDPC1': 'Real_Gross_Domestic_Product', 
        'PCE': 'Personal_Consumption_Expenditures',
        'INDPRO': 'Industrial_Production_Index',
        'RECPROUSM156N': 'Recession_Probabilities',
        # Inflation & Real Rates
        'CPILFESL': 'Core_CPI_(ex_Food/Energy)', 
        'T10YIE': '10-Year_Breakeven_Inflation',
        'DFII10': '10-Year_TIPS_Yield',
        'DGS10': '10-Year_Treasury_Yield',
        # Labour Market
        'UNRATE': 'Unemployment_Rate', 
        'PAYEMS': 'Nonfarm_Payrolls',
        'JTSJOL': 'Job_Openings_(JOLTS)',
        # Rates & Financial Conditions
        'FEDFUNDS': 'Effective_Fed_Funds_Rate',
        'DGS2': '2-Year_Treasury_Yield',
        'T10Y2Y': '10Y-2Y_Treasury_Spread',
        'BAMLH0A0HYM2': 'High_Yield_OAS',
        #Tech/Growth Specific
        'EMVMACROINTEREST': 'Equity_Mkt_Vol:_Macro_News_Tracker',
        'WLEMUINDXD': 'Equity_Market_Uncertainty_Index',
        'MICH': 'U_Mich_Inflation_Expectations',
        #Housing & Consumer Credit
        'HOUST': 'Housing_Starts:_Total',
        'PERMIT': 'New_Private_Housing_Units_Authorized',
        'MORTGAGE30US': '30-Year_Fixed_Mortgage_Rate',
        #Money Supply & Credit
        'M2SL': 'M2_Money_Stock',
        'RRPONTSYD': 'ON_RRP_(Reverse_Repo)_Balance',
        'BOGMBASE': 'Monetary_Base',
        'BUSLOANS': 'Commercial_&_Industrial_Loans',
        #Consumer & Business Sentiment
        'UMCSENT': 'U_Mich_Consumer_Sentiment',
        'CFNAI': 'Chicago_Fed_National_Activity_Index',
        #Additional Volatility & Financial Conditions
        'VIXCLS': 'CBOE_Volatility_Index (VIX)',
        'NFCI': 'Chicago_Fed_National_Financial_Conditions',
        'EMVMACROBROAD': 'Equity_Vol_Tracker:_Broad Macro',
        'T10Y3M': '10Y-3M_Treasury_Spread',
        'BAA10Y': 'Moodys_BAA_Corporate_Bond_Yield',
        #High-Frequency & Forward-Looking
        'WPSFD49207': 'Capacity_Utilization:_Manufacturing',
        'DGS1MO': '1-Month_Treasury_Bill',
        'DCOILWTICO': 'WTI_Crude_Oil_Price'
    }

    data = {}
    
    for id, name in series_dict.items():
        data[id] = fred.get_series(id, 
                                   observation_start=obs_start, 
                                   observation_end=obs_end)

    df = pd.DataFrame(data).ffill().dropna()

    df = df.rename(columns=series_dict)

    print(f"Observation Start/end: {obs_start}/{obs_end} | Shape: {df.shape}\n")
    print(df.head(2))
    print(df.tail(2),"\n")

    df = df.rename_axis("Date")
    df.to_csv(r'..\Resources\Raw_Files\FRED\fred_macro_data.csv', index=True )

In [32]:
def yahoo_fin_fetch_list(tickers,start,end,interval):
    
    if isinstance(tickers, str): #adds single ticker to list
        tickers = [tickers]

    for ticker in tickers:
        data = yf.Ticker(ticker)

        df = data.history(
            start = start,
            end = end,
            interval = interval
        )

        df['Date'] = df.index.date  # Extracts YYYY-MM-DD as date objects
        df.reset_index(drop=True, inplace=True)  # If you want Date as column
        date_col = df.pop('Date')    # remove column
        df.insert(0, 'Date', date_col)  # insert at position 0
        
        print(f"Ticker: {ticker} | Shape: {df.shape}\n")
        print(df.head(2))
        print(df.tail(2),"\n")

        filename = f"..\Resources\Raw_Files\YFINANCE\{ticker}_{interval}_data.csv"
        df.to_csv(filename, index = False)

In [ ]:
# Just a few ETFs for testing to start with to see if everything works and model is accurate enough to proceed withto the full list (beiow)
ticker_list1 = ["QQQ",
                "QQQE",
                "SKY",
                "VOO",
                "IVV",
                "VTI",
                "ITOT"]

# Once we find out everything is working, we can use the full list (below) to see if we can run multiple ETF models successfully
ticker_list2 = [
    'SKY', 'VOO', 'IVV', 'VTI', 'ITOT',    # Core S&P/Total Market
    'QQQ', 'QQQE',                         # Nasdaq/Growth
    'IWM', 'IWF', 'IWD', 'MDY',            # Small/Mid/Growth/Value
    'XLK', 'XLF', 'XLE', 'XLV',             # Tech/Finance/Energy
    'TLT', 'BND', 'HYG',                   # Bonds
    'VEU', 'EFA'                           # International (US-listed)
]

#needed for both API data extractions
start_date = "2020-01-01"
end_date = "2025-12-31"

#Only required for Yahoo Finance data extraction
interval = "1d"


In [27]:
fred_data_extract(start_date,end_date)

Observation Start/end: 2020-01-01/2025-12-31 | Shape: (2190, 46)

                  GDP  CPI_Inflation  Average_price_of_eggs  \
2020-01-03  21751.238        259.127                  1.461   
2020-01-04  21751.238        259.127                  1.461   

            Average_price_of_electricity  Leading_Index_for_US  \
2020-01-03                         0.134                  1.57   
2020-01-04                         0.134                  1.57   

            NBER_Recession_Indicator  Producer_Price_Index  \
2020-01-03                       0.0                 199.3   
2020-01-04                       0.0                 199.3   

            5-Year_Breakeven_Inflation  Natural_Rate__of_Unemployment_(est.)  \
2020-01-03                        1.85                              4.403836   
2020-01-04                        1.85                              4.403836   

            Civilian_Labor_Force_Participation  ...  \
2020-01-03                                63.3  ...   
2020-01

In [33]:
yahoo_fin_fetch_list(ticker_list2, start_date, end_date, interval)

Ticker: SKY | Shape: (1507, 8)

         Date       Open       High    Low      Close  Volume  Dividends  \
0  2020-01-02  31.870001  32.119999  31.43  32.110001  452300        0.0   
1  2020-01-03  31.639999  32.110001  30.85  32.060001  440100        0.0   

   Stock Splits  
0           0.0  
1           0.0  
            Date       Open       High        Low      Close  Volume  \
1505  2025-12-29  85.610001  86.029999  84.669998  85.739998  296800   
1506  2025-12-30  86.150002  86.150002  84.730003  85.410004  268600   

      Dividends  Stock Splits  
1505        0.0           0.0  
1506        0.0           0.0   

Ticker: VOO | Shape: (1507, 9)

         Date        Open        High         Low       Close   Volume  \
0  2020-01-02  271.727504  272.787765  270.859179  272.760345  3142400   
1  2020-01-03  269.835446  271.791423  269.670930  270.767731  3247900   

   Dividends  Stock Splits  Capital Gains  
0        0.0           0.0            0.0  
1        0.0           0.0 